In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import matplotlib.dates as md
import seaborn as sns
import datetime as datetime
import numpy as np
import os

%matplotlib inline

plt.style.use('default')

sns.set(style="whitegrid")

import warnings 
warnings.filterwarnings('ignore')

my_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# ------------- Para uso luego --------------
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sksurv.preprocessing import OneHotEncoder

In [2]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

Limpieza_Clicks = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Clicks.ipynb'))
Limpieza_Auctions = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Auctions.ipynb'))
Limpieza_Installs = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Installs.ipynb'))
Limpieza_Events = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Events.ipynb'))

In [3]:
%run $Limpieza_Clicks
%run $Limpieza_Auctions
%run $Limpieza_Installs
%run $Limpieza_Events

In [4]:
training_set = auctions
training_set.sort_values(ascending=True,by='date',inplace=True)

In [5]:
#Separamos el set de entrenamiento en 5 "ventanas" de 3 días cada una.
ventana1 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-18')) & (training_set['date'] < pd.to_datetime('2019-04-21'))]
ventana2 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-19')) & (training_set['date'] < pd.to_datetime('2019-04-22'))]
ventana3 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-20')) & (training_set['date'] < pd.to_datetime('2019-04-23'))]
ventana4 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-21')) & (training_set['date'] < pd.to_datetime('2019-04-24'))]
ventana5 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-22')) & (training_set['date'] < pd.to_datetime('2019-04-25'))]

In [6]:
list_devices = list(set(training_set['device_id'].tolist()))
df = pd.DataFrame(list_devices) 
df.columns = ['device_id']

In [7]:
ventana1 = pd.merge(df,ventana1,on='device_id',how='left')
ventana2 = pd.merge(df,ventana2,on='device_id',how='left')
ventana3 = pd.merge(df,ventana3,on='device_id',how='left')
ventana4 = pd.merge(df,ventana4,on='device_id',how='left')
ventana5 = pd.merge(df,ventana5,on='device_id',how='left')

### Explico lo que se busca hacer
En la siguiente linea lo que se busca es para los dispositivos que no tienen reg en esta venatana, se les coloca una fecha de la ventana siguiente, de forma que tengan el tiempo maximo de la ventana luego al calcular el st/sc
#### ventana1['date'].fillna(value=pd.to_datetime("2019-04-21"),inplace=True)

Luego seteo una fecha de inicio de la ventana para calcular los segs de cada registro contra esta. de forma que si el dispositivo aparece, es a X segundos del inicio de esta ventana. Luego hago el promedio de los segs de cada registro para cada dispositivo, generando un nuevo df al cual luego se le agregaran los features.
#### fecha_inicio = pd.to_datetime("2019-04-18")
#### ventana1['date_seconds'] = (ventana1['date']-fecha_inicio).dt.total_seconds()
#### ventana1_st = ventana1[['device_id','date_seconds']].groupby('device_id').mean().reset_index()
#### ventana1_st.columns = ['device_id','st']

In [8]:
ventana1['date'].fillna(value=pd.to_datetime("2019-04-21"),inplace=True)
fecha_inicio = pd.to_datetime("2019-04-18")
ventana1['date_seconds'] = (ventana1['date']-fecha_inicio).dt.total_seconds()
ventana1_st = ventana1[['device_id','date_seconds']].groupby('device_id').mean().reset_index()
ventana1_st.columns = ['device_id','st']

In [9]:
ventana2['date'].fillna(value=pd.to_datetime("2019-04-22"),inplace=True)
fecha_inicio = pd.to_datetime("2019-04-19")
ventana2['date_seconds'] = (ventana2['date']-fecha_inicio).dt.total_seconds()
ventana2_st = ventana2[['device_id','date_seconds']].groupby('device_id').mean().reset_index()
ventana2_st.columns = ['device_id','st']

In [10]:
ventana3['date'].fillna(value=pd.to_datetime("2019-04-23"),inplace=True)
fecha_inicio = pd.to_datetime("2019-04-20")
ventana3['date_seconds'] = (ventana3['date']-fecha_inicio).dt.total_seconds()
ventana3_st = ventana3[['device_id','date_seconds']].groupby('device_id').mean().reset_index()
ventana3_st.columns = ['device_id','st']

In [11]:
ventana4['date'].fillna(value=pd.to_datetime("2019-04-24"),inplace=True)
fecha_inicio = pd.to_datetime("2019-04-21")
ventana4['date_seconds'] = (ventana4['date']-fecha_inicio).dt.total_seconds()
ventana4_st = ventana4[['device_id','date_seconds']].groupby('device_id').mean().reset_index()
ventana4_st.columns = ['device_id','st']

In [12]:
ventana5['date'].fillna(value=pd.to_datetime("2019-04-25"),inplace=True)
fecha_inicio = pd.to_datetime("2019-04-22")
ventana5['date_seconds'] = (ventana5['date']-fecha_inicio).dt.total_seconds()
ventana5_st = ventana5[['device_id','date_seconds']].groupby('device_id').mean().reset_index()
ventana5_st.columns = ['device_id','st']

In [13]:
ventana1_st.sample(10)

,device_id,st
177542,3447451428440269355,126130.650969
266128,5165660681934737933,199125.645031
427273,8293718336415615699,34707.670535
446509,8668195199834708441,259200.000000
32823,640412101775753849,259200.000000
228750,4439500221329079685,259200.000000
55604,1082771975484126137,174350.177080
290656,5641285281654433558,259200.000000
404752,7857143308949027478,259200.000000
332483,6448452239984892409,134166.630051
